# Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
from typing import List
from dotenv import load_dotenv

from lib.vector_db import VectorStore, VectorStoreManager, CorpusLoaderService

In [3]:
load_dotenv()

False

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### VectorDB Instance

#### How VectorStoreManager and CorpusLoaderService work
- **VectorStoreManager**: wraps a Chroma client and configures the OpenAI embedding function. It uses `OPENAI_BASE_URL` (defaulting to Vocareum) and `CHROMA_EMBED_MODEL` for embeddings, and exposes `get_or_create_store(name)` to return a Chroma collection wrapped as a `VectorStore`.
- **VectorStore (wrapper)**: provides `add(...)`, `query(...)`, and `get(...)` on top of a Chroma collection, handling conversion from our `Document`/`Corpus` types to Chroma’s batch format and including metadatas.
- **CorpusLoaderService**: orchestrates ingestion. For JSON, `load_json_dir(store_name, path)` builds a `Corpus` of `Document`s via `JSONLoader` (content = normalized summary; metadata = full JSON), then calls `store.add(corpus)`. Each insert triggers automatic embedding via the collection’s embedding function, so future `query(...)` runs cosine-similarity search over those vectors.

This separation lets you swap loaders (PDF/JSON/CSV) without changing vector DB code, and keeps Chroma + embeddings configuration centralized in one place.


In [5]:
db = VectorStoreManager(OPENAI_API_KEY)
db

VectorStoreManager():<chromadb.api.client.Client object at 0x1146f42f0>

### Collection

In [6]:
# loader function looks as follows
def load_json_dir(self, store_name: str, json_path_or_dir: str) -> VectorStore:
    """
    Load JSON file(s) into a vector store.

    If a directory is provided, all *.json files are loaded. Otherwise, a
    single JSON file is loaded. Each JSON record becomes one Document with
    flattened content and full metadata preserved.

    Args:
        store_name (str): Name of the vector store to create or use
        json_path_or_dir (str): Path to a JSON file or directory of JSON files

    Returns:
        VectorStore: The vector store containing the loaded JSON content
    """
    store = self.manager.get_or_create_store(store_name)
    print(f"VectorStore `{store_name}` ready!")

    loader = JSONLoader(json_path_or_dir)
    corpus = loader.load()
    store.add(corpus)
    print(f"JSON loaded from `{json_path_or_dir}`!")

    return store

In [7]:
loader_service = CorpusLoaderService(db)
games_store = loader_service.load_json_dir(
    store_name="games_market",
    json_path_or_dir="./games"
)

VectorStore `games_market` ready!
JSON loaded from `./games`!


### Demo

The following is an example of a query on the Vector Store Database.

In [8]:
query_text = "best racing game on PlayStation around 1997"
results = games_store.query(
    query_texts=[query_text],
    n_results=5,
    where={"Platform": "PlayStation 1"} 
)

In [9]:
docs = results["documents"][0]
metas = results["metadatas"][0]
dists = results["distances"][0]

for doc, meta, dist in zip(docs, metas, dists):
    print(f"similarity≈{1 - dist:.3f} | {meta.get('Name')} ({meta.get('YearOfRelease')}) [{meta.get('Platform')}]")
    print(doc[:120], "...\n")

similarity≈0.743 | Gran Turismo (1997) [PlayStation 1]
Name: Gran Turismo
Platform: PlayStation 1
Genre: Racing
Publisher: Sony Computer Entertainment
Year: 1997
Description:  ...

